In [1]:
import uproot
import numpy as np
import sys

In [2]:
# Retrieve flags
file_data = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
file_data['fq1rpcflg'].interpretation
flags = file_data.arrays('fq1rpcflg')['fq1rpcflg']

fq1rpcflg_1 = flags[:, 0, 1]
fq1rpcflg_2 = flags[:, 0, 2]

In [3]:
print(np.array(fq1rpcflg_1).shape)

(1200000,)


In [6]:
file_data.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
cluster_ncand        | int32_t                  | AsDtype('>i4')
cluster_tstart       | float[]                  | AsJagged(AsDtype('>f4'))
cluster_tend         | float[]                  | AsJagged(AsDtype('>f4'))
cluster_nhits        | int32_t[]                | AsJagged(AsDtype('>i4'))
cluster_totq         | float[]                  | AsJagged(AsDtype('>f4'))
cluster_goodflag     | int32_t[]                | AsJagged(AsDtype('>i4'))
cluster_npeaks       | int32_t[][6]             | AsJagged(AsDtype("('>i4', (...
cluster_ipeak        | int32_t[][6][10]         | AsJagged(AsDtype("('>i4', (...
cluster_timeofpeak   | float[][6][10]           | AsJagged(AsDtype("('>f4', (...
muechk_ncand         | int32_t[6]               | AsDtype("('>i4', (6,))")
muechk_toya          | float[6]                 | AsDtype("('>f4', (6,))")
muech